Código sin landmarks

In [ ]:
import os
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np

# Configuración de Mediapipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils  # Opcional, para dibujar los landmarks (depuración)

# Ruta del dataset (ajústala según tu estructura)
dataset_dir = '..\\ASL\\asl_dataset\\'  # Cambia esto a la ruta de tu dataset
letters = sorted(os.listdir(dataset_dir))  # Lista de carpetas por letra

# Lista para almacenar los datos procesados
data = []

# Procesar cada carpeta (letra) y las imágenes dentro de ella
for letter in letters:
    letter_dir = os.path.join(dataset_dir, letter)
    
    # Verificar que sea un directorio válido
    if not os.path.isdir(letter_dir):
        print(f"Advertencia: {letter_dir} no es un directorio válido. Saltando...")
        continue
    
    print(f"Procesando letra: {letter}")  # Depuración: letra actual

    for image_file in os.listdir(letter_dir):
        image_path = os.path.join(letter_dir, image_file)
        print(f"Procesando imagen: {image_path}")  # Depuración: imagen actual
        
        # Leer la imagen
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: No se pudo leer la imagen {image_path}. Saltando...")
            continue
        
        # Convertir la imagen a RGB (requerido por Mediapipe)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Procesar la imagen con Mediapipe
        results = hands.process(image_rgb)
        
        # Si se detectan landmarks, extraerlos
        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0]
            
            # Extraer coordenadas (x, y, z) de los landmarks
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.append([lm.x, lm.y, lm.z])
            landmarks_flatten = np.array(landmarks).flatten()  # Aplanar los datos
            
            # Guardar los landmarks y la etiqueta (letra)
            data.append(landmarks_flatten.tolist() + [letter])
        else:
            print(f"Advertencia: No se detectaron landmarks en {image_path}. Saltando...")

# Cerrar Mediapipe
hands.close()

# Crear el DataFrame y guardar como CSV
columns = [f"x{i}" for i in range(63)] + ["label"]  # 63 coordenadas (x, y, z) + etiqueta
df = pd.DataFrame(data, columns=columns)

# Guardar el CSV
output_csv = "asl_landmarks.csv"
df.to_csv(output_csv, index=False)
print(f"Dataset procesado guardado en {output_csv}")


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd

# Cargar el dataset procesado
data = pd.read_csv('asl_landmarks.csv')

# Separar las características (X) y las etiquetas (y)
X = data.iloc[:, :-1].values  # Coordenadas de los landmarks
y = data.iloc[:, -1].values   # Etiquetas (letras)

# Dividir los datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar un modelo SVM
model = SVC(kernel='linear', probability=True)
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.2f}")

# Guardar el modelo entrenado para su uso posterior
import joblib
joblib.dump(model, "svm_asl_model.pkl")

Precisión del modelo: 0.89


['asl_model.pkl']